Funktioniert nicht

In [1]:
from openprompt import PromptDataLoader, PromptForClassification
from openprompt.data_utils import InputExample, InputFeatures
from openprompt.plms import load_plm
from openprompt.prompts import ManualTemplate, ManualVerbalizer, ManualTemplate
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
import torch
import pandas as pd
from transformers import BertForSequenceClassification

g:\python\lib\site-packages\transformers\generation_utils.py:27: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  FutureWarning,


In [2]:
plm = BertForSequenceClassification.from_pretrained("checkp/saved-model/")

In [3]:
plm, tokenizer, _, WrapperClass = load_plm("bert", "bert-large-uncased")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Prepare Data

Load dataset

In [4]:
# load data
pokemon_descriptions = load_dataset('../data/dataset/', delimiter=';')
NUM_CLASSES = np.unique(pokemon_descriptions['train']['labels'])

Using custom data configuration dataset-294e9b13f49dafc6
Found cached dataset csv (C:/Users/fst/.cache/huggingface/datasets/csv/dataset-294e9b13f49dafc6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Make split

In [5]:
# train test split
split_pokemon_descriptions = pokemon_descriptions['train'].train_test_split(
    test_size=0.2, shuffle=True)

Generate InputExamples from existing dataset

In [6]:
dataset = {}
for split in ['train']:
    dataset[split] = []
    for sample in split_pokemon_descriptions[split]:
        input_example = InputExample(text_a = sample['text'], label=int(sample['labels']))
        dataset[split].append(input_example)

Create template

In [7]:
promptTemplate = ManualTemplate(
    text = '{"placeholder":"text_a"} It is {"mask"}',
    tokenizer = tokenizer,
)

wrapped_example = promptTemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': "Mega Beedril's arms secrete fast acting poison, used to immobilize foes, then finishes them off with its long acting poison from its abdomen", 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' It is', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}], {'label': 14}]


Create verbalizer

In [8]:
mappings = pd.read_csv('../data/pokemon_mapping.csv')
name_to_label_dict = mappings[["name","index"]].set_index('index').to_dict()["name"]

In [9]:
promptVerbalizer = ManualVerbalizer(
    classes = NUM_CLASSES,
    label_words = name_to_label_dict,
    tokenizer = tokenizer,
)

Create dataloader

In [19]:
train_dataloader = PromptDataLoader(
  dataset=dataset["train"],
  template=promptTemplate, 
  tokenizer=tokenizer,
  tokenizer_wrapper_class=WrapperClass, 
  shuffle=True,
  truncate_method="head",
  decoder_max_length=3,
  batch_size=2,
  teacher_forcing=False,
  predict_eos_token=False,
  max_seq_length=128,
)

RuntimeError: Either wrapped_tokenizer or tokenizer_wrapper_class should be specified.

Create model

In [16]:
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
    freeze_plm= False
)
promptModel= promptModel.cuda()

In [17]:
epochs = 3
no_decay = ['bias', 'layer_norm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in promptModel.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in promptModel.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters)
loss_func = torch.nn.CrossEntropyLoss()

In [18]:
for epoch in range(epochs):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        inputs = inputs.cuda()
        logits = promptModel(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()

        if step %100 == 0:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

IndexError: tuple index out of range

In [ ]:
#torch.save(promptModel.state_dict(),"checkp/bert_large_trained_model.cp")

### Versuch 1

- True = [133, 12, 46, 44, 70, 0, 101, 134, 113, 109, 136, 38, 91, 143, 59, 110, 127, 116, 98, 80, 149, 48, 46]
- Predicted = [ 94,  78,  94,  94,  78, 105,  94,  94,  94,  94,  54,  94,  94,  78, 98,  94,  78,  94,  94, 105,  28,  78,  94]

In [35]:
dataset_test = {}
for split in ['test']:
    dataset_test[split] = []
    for sample in split_pokemon_descriptions[split]:
        input_example = InputExample(text_a = sample['text'], label=int(sample['labels']))
        dataset_test[split].append(input_example)

In [36]:
test_dataloader = PromptDataLoader(dataset=dataset_test["test"], template=promptTemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=250, decoder_max_length=3,
    batch_size=1,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 2158it [00:02, 1001.31it/s]


In [37]:
allpreds = []
alllabels = []
for step, inputs in enumerate(test_dataloader):
    inputs = inputs.cuda()
    logits = promptModel(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cuda().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cuda().tolist())

acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
print(acc)

IndexError: tuple index out of range

Same without store and load states

In [38]:
inference_text = "Intelligent and can use an incredible variety of moves. Its mythical and rarely seen. While many think it is first pokemon coming into existing. It is rarely seen and many are scared of it"

inference_input = InputExample(text_a = inference_text)

inference_dataloader = PromptDataLoader(dataset=[inference_input], template=promptTemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=250, decoder_max_length=3,
    batch_size=1,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 1it [00:00, 500.22it/s]


In [39]:
for index, inputs in enumerate(inference_dataloader):
    logits = promptModel(inputs)

predicted_class = torch.argmax(logits, dim=-1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [ ]:
print(f"Input: {inference_text}")
print(f"Input with template: {inference_input}")
print(f"The pokemon is: {name_to_label_dict[int(predicted_class)]}")

Input: Intelligent and can use an incredible variety of moves. Its mythical and rarely seen. While many think it is first pokemon coming into existing. It is rarely seen and many are scared of it
Input with template: {
  "guid": null,
  "label": null,
  "meta": {},
  "text_a": "Intelligent and can use an incredible variety of moves. Its mythical and rarely seen. While many think it is first pokemon coming into existing. It is rarely seen and many are scared of it",
  "text_b": "",
  "tgt_text": null
}

The pokemon is: Alakazam


# Inferenz

In [ ]:
inference_text = "Intelligent and can use an incredible variety of moves. Its mythical and rarely seen. While many think it is first pokemon coming into existing. It is rarely seen and many are scared of it"

inference_input = InputExample(text_a = inference_text)

inference_dataloader = PromptDataLoader(dataset=[inference_input], template=promptTemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=250, decoder_max_length=3,
    batch_size=1,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 1it [00:00, 642.21it/s]


In [ ]:
promptLoadedModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
    freeze_plm= True
)

In [ ]:
promptLoadedModel.load_state_dict(state_dict=torch.load("checkp/bert_large_fine_tuning_trained_model.cp"))

<All keys matched successfully>

In [ ]:
for index, inputs in enumerate(inference_dataloader):
    logits = promptLoadedModel(inputs)

predicted_class = torch.argmax(logits, dim=-1)

In [ ]:
print(f"Input: {inference_text}")
print(f"Input with template: {inference_input}")
print(f"The pokemon is: {name_to_label_dict[int(predicted_class)]}")

Input: Intelligent and can use an incredible variety of moves. Its mythical and rarely seen. While many think it is first pokemon coming into existing. It is rarely seen and many are scared of it
Input with template: {
  "guid": null,
  "label": null,
  "meta": {},
  "text_a": "Intelligent and can use an incredible variety of moves. Its mythical and rarely seen. While many think it is first pokemon coming into existing. It is rarely seen and many are scared of it",
  "text_b": "",
  "tgt_text": null
}

The pokemon is: Alakazam


Clear GPU Mem

In [73]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()